In [6]:
!pip install s3fs

Defaulting to user installation because normal site-packages is not writeable


In [27]:
import json
import pandas as pd
from time import sleep
from json import dumps, loads
from kafka import KafkaConsumer

# Inorder to enable sending to s3 bucket you need to
# - Setup s3 bucket 
# - Configure AWS on local device using the AWS CLI
from s3fs import S3FileSystem

In [28]:
"""
 - Initializing the Consumer class 
 - Connecting to the kafka bootstrap server
 - Decoding the data
"""
consumer = KafkaConsumer('order_transactions',
                        bootstrap_servers=['0.0.0.0:9092'],
                        value_deserializer=lambda x: loads(x.decode('utf-8')))

In [16]:
# # Consuming the data
# for c in consumer:
#     print(c.value)

In [33]:
# Initialize S3 client
s3 = S3FileSystem()

# Specify the S3 bucket and prefix
s3_bucket = 'kafka-stream-transactions-stan'
s3_prefix = 'transactions'

In [35]:
# Iterate over messages
for count, message in enumerate(consumer):
    
    # Create a unique S3 key for each message
    s3_key = f"{s3_prefix}_{count}.json"

    # Dump the message value to a JSON file and upload to S3
    with s3.open(f"s3://{s3_bucket}/{s3_key}", 'w') as s3_file:
        print(f"Starting processing message {count}.")
        try:
            json.dump(message.value, s3_file)
        except Exception as e:
            print(f"Error while dumping JSON: {e}")


    print(f"Done processing message {count}.")
        